In [1]:
# Uisng COSINE SIMILARITY FOR TEXT SUMMARIZING./

In [2]:
from goose3 import Goose
from nltk import sent_tokenize
import string
import nltk
import pint
import numpy as np 
ureg=pint.UnitRegistry()
stopwords = nltk.corpus.stopwords.words('english')
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd  
import networkx as nx # it is used to generate graphs
from nltk.cluster.util import cosine_distance# for cosine similarity based outputs

In [3]:
def get_data():
    while True:
        print("1. For Url Based summarizing using wikipedia\n")
        print("2. For Input Text from your keyboard\n ")
        print("3.QUIT\n")
        try:
            choice = input("Enter your choice\n") 
            if int(choice) == 1:
                url=input("Enter the word to be searched on wikipedia \n") 
                url =f'https://en.wikipedia.org/wiki/{url}'
                try:
                    text=Goose().extract(url).cleaned_text
                    return text
                except:
                    print(" Wrong URL format")
            elif int(choice) ==2:
                text = input("Input the copied text")
                return text
            elif int(choice) ==3:
                print("bye\n") 
                break
            else:
                print(" wrong choice enetered, Exiting the Program \n")
                break
        except:
            print(" Input Not a number, Quitting the program")
            break

In [4]:
# Because w eneed sentences for this we are not going to remove the . which seprates the sentences 
punct=string.punctuation
def preprocess_data(text):
    text=text.replace('\n','')
    text=text.lower() 
    text=sent_tokenize(text)
    save_text = text # original text
    translator = str.maketrans('','', punct+ '0123456789'+'•°'+'─') 
    clean_text=[]
    for i in text:
        a=i.translate(translator)
        clean_text.append(a) 
    # so we have punctuation free etxt here, now we need to remove stopwords and units from it.
    cleaner_text=[]
    for i in clean_text:
        l=[]
        for j in i.split():
            if j not in stopwords+list(ureg):
                l.append(j)
        cleaner_text.append(' '.join(l)) 
# cleaner text have no stopwords and most of the symbols of the data have been removed there fore we can use this cleaner text
# for getting the scores for our data.
    text = ' '.join(cleaner_text)    
    tokens=text.split()
    print(f"Total data that we are taking in consideration is {len(tokens)}\n") 
    print(f" Unique words in our data is {len(set(tokens))}\n")
    tokenizer= Tokenizer()
    tokenizer.fit_on_texts(tokens)
    tokenized_text=tokenizer.texts_to_sequences(tokens)# Giving the clean text number represenattion
    # but for frequency based summariziation we only need the frequency count from tokenizer.
    freq_dataframe = pd.DataFrame(list(tokenizer.word_counts.items()),columns=['Words','Count'])
    # we need the frequency, with their word counts 
    return cleaner_text,set(tokens),save_text
# here we are sending the frequency of accurance of each word in the data.

In [6]:
text1=get_data()

1. For Url Based summarizing using wikipedia

2. For Input Text from your keyboard
 
3.QUIT

Enter your choice
1
Enter the word to be searched on wikipedia 
chocolate


In [7]:
cleaner_text,unique_words,original_text=preprocess_data(text1)

Total data that we are taking in consideration is 3816

 Unique words in our data is 1762



In [8]:
list(unique_words) # we have all the unique words now we want to assign the values to sentences based of the count of these words in the 
# sentence to make it a vector.

['energy',
 'mexican',
 'butterfat',
 'manually',
 'order',
 'formationschocolate',
 'reaching',
 'life',
 'quantity',
 'derived',
 'conformation',
 'preolmec',
 'destroy',
 'supplywith',
 'coffee',
 'consumer',
 'smarties',
 'influence',
 'trees',
 'americas',
 'offer',
 'mostly',
 'without',
 'ingredients',
 'red',
 'easter',
 'highquality',
 'manufacturers',
 'heartburn',
 'vanilla',
 'highlands',
 'directly',
 'chocol',
 'shell',
 'cakes',
 'truffle',
 'university',
 'crystallization',
 'frozen',
 'pilot',
 'willy',
 'business',
 'climates',
 'μm',
 'mouthfeelthe',
 'crisp',
 'polymorphic',
 'florida',
 'emulsifier',
 'stick',
 'uk',
 'conche',
 'nicaragua',
 'campaigning',
 'consuming',
 'commonly',
 'found',
 'extremes',
 'warmed',
 'allspice',
 'laborers',
 'residues',
 'subsequently',
 'highest',
 'vii',
 'finished',
 'labelled',
 'large',
 'unfair',
 'crew',
 'increase',
 'gas',
 'moving',
 'glossy',
 'referred',
 'drinking',
 'minerals',
 'drank',
 'using',
 'abilities',
 'in

In [10]:
# Bag of words approach..
vectors_list=[] 
for i in range(len(cleaner_text)):
    vector=[0]*len(unique_words)
    for j in cleaner_text[i].split():
        if j in list(unique_words):
            vector[list(unique_words).index(j)] +=1
    vectors_list.append([vector])

In [11]:
if 2 in vectors_list[0]: # this vector only belongs to one sentence that is based of the all the unique words in the sentence 
    print('yes')

In [12]:
if 3 in vector:
    print('yes')
    # this proves that teh vectors are created successfluuly based off their count in the sentence.

yes


In [13]:
1-cosine_distance(vectors_list[0][0],vectors_list[0][0])
# if the 2 vectors ARE TEH same we get 0 vector as they are just overlapping on each other and using this way we need to 
# calculate the score based of each sentnece. 
# I am just going to use 1 - so that similar sentences are 1, giving higer score to those sentences that are equal

1.0

In [14]:
len(vectors_list[0][0])

1762

In [15]:
# we need to calculate a simiarity matrix for every sentnec with every other sentnec and we will store it in a matrix
# this function takes a lot of time to run 
def calculate_score():
    similarity_matrix=np.zeros((len(cleaner_text),len(cleaner_text)))
    for i in range(len(cleaner_text)):
        for j in range(len(cleaner_text)):
            score=1-cosine_distance(vectors_list[i][0],vectors_list[j][0])
            similarity_matrix[i][j]=score
    return similarity_matrix

In [16]:
sim=calculate_score()

In [17]:
sim
# we have gotten similarity scores for each of the sentence with other sentence.

array([[1.        , 0.1767767 , 0.07905694, ..., 0.        , 0.        ,
        0.1       ],
       [0.1767767 , 1.        , 0.0745356 , ..., 0.11785113, 0.        ,
        0.0942809 ],
       [0.07905694, 0.0745356 , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.11785113, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.1       , 0.0942809 , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [18]:
# we will make use of pagerank formulae which basically tells us which seentence is more important based of teh similarity matrix
# This formule tells us that the quality of a sentnce is basically determined by the number of sentence connected to iit 

In [19]:
similarity_graph=nx.from_numpy_array(sim)

In [20]:
scores=nx.pagerank(similarity_graph) # here we get the scores based of each sentnce.Uinsg these scores we can now have the summary.

In [21]:
final_df=pd.DataFrame(original_text,columns=['Text'])

In [22]:
final_df['Scores']=scores.values()

In [23]:
final_df_Sorted=final_df.sort_values(by='Scores',ascending=False).reset_index(drop=True)

In [24]:
final_df_Sorted # We can now print the sentences

,Text,Scores
0,[69]according to canadian food and drug regula...,0.008177
1,"[132]chocolate is sold in chocolate bars, whic...",0.008167
2,"white chocolate contains cocoa butter, sugar, ...",0.007587
3,a sample cup is filled with the chocolate and ...,0.007408
4,these include baking chocolate (often unsweete...,0.007311
...,...,...
273,art and myth of the ancient maya.,0.001218
274,this provides the best appearance and texture ...,0.001210
275,"heating the working areas of the table-mill, a...",0.001189
276,"[116] mars incorporated, a large privately own...",0.001174


In [25]:
def summary(per=0.5):
    m=0
    for i in range(len(final_df_Sorted)):
        if m <=per:
            m+=final_df_Sorted['Scores'][i]
            print(final_df_Sorted['Text'][i])

In [26]:
summary(0.05) # WORKING

[69]according to canadian food and drug regulations,[70] a "chocolate product" is a food product that is sourced from at least one "cocoa product" and contains at least one of the following: "chocolate, bittersweet chocolate, semi-sweet chocolate, dark chocolate, sweet chocolate, milk chocolate, or white chocolate".
[132]chocolate is sold in chocolate bars, which come in dark chocolate, milk chocolate and white chocolate varieties.
white chocolate contains cocoa butter, sugar, and milk, but no cocoa solids.chocolate is one of the most popular food types and flavors in the world, and many foodstuffs involving chocolate exist, particularly desserts, including cakes, pudding, mousse, chocolate brownies, and chocolate chip cookies.
a sample cup is filled with the chocolate and placed in the unit which then displays or prints the results.two classic ways of manually tempering chocolate are:• working the molten chocolate on a heat-absorbing surface, such as a stone slab, until thickening ind